# Question B1 (15 marks)

Real world datasets often have a mix of numeric and categorical features – this dataset is one example. To build models on such data, categorical features have to be encoded or embedded.

PyTorch Tabular is a library that makes it very convenient to build neural networks for tabular data. It is built on top of PyTorch Lightning, which abstracts away boilerplate model training code and makes it easy to integrate other tools, e.g. TensorBoard for experiment tracking.

For questions B1 and B2, the following features should be used:   
- **Numeric / Continuous** features: dist_to_nearest_stn, dist_to_dhoby, degree_centrality, eigenvector_centrality, remaining_lease_years, floor_area_sqm
- **Categorical** features: month, town, flat_model_type, storey_range



---



In [1]:
!pip install pytorch_tabular[extra]

  Using cached einops-0.7.0-py3-none-any.whl.metadata (13 kB)
  Using cached rich-13.7.1-py3-none-any.whl.metadata (18 kB)
     ---------------------------------------- 0.0/117.0 kB ? eta -:--:--
     -------------------- ------------------- 61.4/117.0 kB ? eta -:--:--
     --------------------------- ----------- 81.9/117.0 kB 1.1 MB/s eta 0:00:01
     ------------------------------------ 117.0/117.0 kB 970.9 kB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached markdown_it_py-3.0.0-py3-none-any.whl.metadata (6.9 kB)
  Using cached mdurl-0.1.2-py3-none-any.whl.metadata (1.6 kB)
  Using cached aiosignal-1.3.1-py3-none-any.whl.metadata (4.0 kB)
   ---------------------------------------- 0.0/44.5 kB ? eta -:--:--
   ---------------------------------------- 0.0/44.5 kB ? eta -:--:--
   --------------------------- ------------ 30.7/44.5 kB ? eta -:--:--
   --------------------------- ------------ 30.7/44.5 kB ?

In [35]:
SEED = 42

import os

import random
random.seed(SEED)

import numpy as np
np.random.seed(SEED)

import pandas as pd

from pytorch_tabular import TabularModel
from pytorch_tabular.models import CategoryEmbeddingModelConfig
from pytorch_tabular.config import (
    DataConfig,
    OptimizerConfig,
    TrainerConfig,
)

1.Divide the dataset (‘hdb_price_prediction.csv’) into train, validation and test sets by using entries from year 2019 and before as training data, year 2020 as validation data and year 2021 as test data.
**Do not** use data from year 2022 and year 2023.



In [55]:
df = pd.read_csv('hdb_price_prediction.csv')
df.head()

,month,year,town,full_address,nearest_stn,dist_to_nearest_stn,dist_to_dhoby,degree_centrality,eigenvector_centrality,flat_model_type,remaining_lease_years,floor_area_sqm,storey_range,resale_price
0,1,2017,ANG MO KIO,406 ANG MO KIO AVENUE 10,Ang Mo Kio,1.007264,7.006044,0.016807,0.006243,"2 ROOM, Improved",61.333333,44.0,10 TO 12,232000.0
1,1,2017,ANG MO KIO,108 ANG MO KIO AVENUE 4,Ang Mo Kio,1.271389,7.983837,0.016807,0.006243,"3 ROOM, New Generation",60.583333,67.0,01 TO 03,250000.0
2,1,2017,ANG MO KIO,602 ANG MO KIO AVENUE 5,Yio Chu Kang,1.069743,9.090700,0.016807,0.002459,"3 ROOM, New Generation",62.416667,67.0,01 TO 03,262000.0
3,1,2017,ANG MO KIO,465 ANG MO KIO AVENUE 10,Ang Mo Kio,0.946890,7.519889,0.016807,0.006243,"3 ROOM, New Generation",62.083333,68.0,04 TO 06,265000.0
4,1,2017,ANG MO KIO,601 ANG MO KIO AVENUE 5,Yio Chu Kang,1.092551,9.130489,0.016807,0.002459,"3 ROOM, New Generation",62.416667,67.0,01 TO 03,265000.0


In [56]:
# YOUR CODE HERE
# Drop full_address, nearest_stn, Year (2022, 2023)
# predict resale_price
# Continuous features: dist_to_nearest_stn, dist_to_dhoby, degree_centrality, eigenvector_centrality, remaining_lease_years, floor_area_sqm
# Categorical features: month, town, flat_model_type, storey_range
print(f"df length before dropping 2022-23: {len(df)}")
df = df[df.year != 2022]
df = df[df.year != 2023]
print(f"df length after dropping 2022-23: {len(df)}")

df = df.drop(['full_address', 'nearest_stn'], axis=1)
# print(f"df:\n{df}")

train = df[df.year < 2020]
val = df[df.year == 2020]
test = df[df.year == 2021]

# print(f"test:\n{test}")

continuous_cols = ["dist_to_nearest_stn", "dist_to_dhoby", "degree_centrality", "eigenvector_centrality", "remaining_lease_years", "floor_area_sqm"]
categorical_cols = [ "month", "town", "flat_model_type", "storey_range"]

df length before dropping 2022-23: 159553
df length after dropping 2022-23: 116427


2.Refer to the documentation of **PyTorch Tabular** and perform the following tasks: https://pytorch-tabular.readthedocs.io/en/latest/#usage
- Use **[DataConfig](https://pytorch-tabular.readthedocs.io/en/latest/data/)** to define the target variable, as well as the names of the continuous and categorical variables.
- Use **[TrainerConfig](https://pytorch-tabular.readthedocs.io/en/latest/training/)** to automatically tune the learning rate. Set batch_size to be 1024 and set max_epoch as 50.
- Use **[CategoryEmbeddingModelConfig](https://pytorch-tabular.readthedocs.io/en/latest/models/#category-embedding-model)** to create a feedforward neural network with 1 hidden layer containing 50 neurons.
- Use **[OptimizerConfig](https://pytorch-tabular.readthedocs.io/en/latest/optimizer/)** to choose Adam optimiser. There is no need to set the learning rate (since it will be tuned automatically) nor scheduler.
- Use **[TabularModel](https://pytorch-tabular.readthedocs.io/en/latest/tabular_model/)** to initialise the model and put all the configs together.

In [57]:
continuous_cols = ["dist_to_nearest_stn", "dist_to_dhoby", "degree_centrality", "eigenvector_centrality", "remaining_lease_years", "floor_area_sqm"]
categorical_cols = [ "month", "town", "flat_model_type", "storey_range"]
target = ['resale_price']

# 1. Define the DataConfig
data_config = DataConfig(
    target= target,
    continuous_cols=continuous_cols,
    categorical_cols=categorical_cols,
)

# 2. Define the TrainerConfig
trainer_config = TrainerConfig(
    auto_lr_find=True, 
    batch_size=1024, 
    max_epochs=50, 
)

# 3. Define the CategoryEmbeddingModelConfig
model_config = CategoryEmbeddingModelConfig(
    task="regression",
    layers="50"  # One hidden layer with 50 neurons.
)

# 4. Create the OptimizerConfig
optimizer_config = OptimizerConfig()

# 5. Create the TabularModel
tabular_model = TabularModel(
    data_config=data_config,
    model_config=model_config,
    optimizer_config=optimizer_config,
    trainer_config=trainer_config,
)

#6. Train the tabular model
tabular_model.fit(train=train, validation=val)
result = tabular_model.evaluate(test)
pred_df = tabular_model.predict(test)

2024-04-21 01:42:35,620 - {pytorch_tabular.tabular_model:140} - INFO - Experiment Tracking is turned off

Seed set to 42


2024-04-21 01:42:35,655 - {pytorch_tabular.tabular_model:524} - INFO - Preparing the DataLoaders

2024-04-21 01:42:35,690 - {pytorch_tabular.tabular_datamodule:499} - INFO - Setting up the datamodule for          
regression task

2024-04-21 01:42:35,882 - {pytorch_tabular.tabular_model:574} - INFO - Preparing the Model: CategoryEmbeddingModel

...

Output()

2024-04-21 01:43:17,300 - {pytorch_tabular.tabular_model:663} - INFO - Training the model completed

2024-04-21 01:43:17,305 - {pytorch_tabular.tabular_model:1489} - INFO - Loading the best model

Output()

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_loss         │       6465972736.0        │
│  test_mean_squared_error  │       6465972736.0        │
└───────────────────────────┴───────────────────────────┘

3.Report the test RMSE error and the test R2 value that you obtained.



In [58]:
pred_df.head()

,resale_price_prediction
87370,136601.484375
87371,166652.906250
87372,299092.687500
87373,294914.812500
87374,266336.750000


In [59]:
# YOUR CODE & RESULT HERE
from sklearn.metrics import r2_score

print(f"{result[0]['test_mean_squared_error']}")
mse = np.sqrt(float(result[0]['test_mean_squared_error']))
pred = pred_df["resale_price_prediction"].values.tolist()
rsquared = r2_score(y, pred)
print(f"test loss: {result[0]['test_loss']}")
print(f"RMSE: {mse}")
print(f"R Squared: {rsquared}")

6465972736.0


NameError: name 'y' is not defined

4.Print out the corresponding rows in the dataframe for the top 25 test samples with the largest errors. Identify a trend in these poor predictions and suggest a way to reduce these errors.



In [41]:
pred_df['error'] = abs(pred_df['resale_price'] - pred_df['resale_price_prediction'])
# print(f"pred_df:\n{pred_df}")
sorted_df = pred_df.sort_values("error", ascending=False)
error_df = sorted_df.head(25)
error_df

KeyError: "['full_address', 'nearest_stn', 'year'] not found in axis"